# Chapter Fifth - Data Processing with Optimus 

## Why Optimus exist? 

Create Optimus with pandas as engine

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../Optimus")

In [3]:
from optimus import Optimus 
op = Optimus("pandas") 

C:\Users\argenisleon\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\argenisleon\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\argenisleon\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


pandas


In [4]:
df = op.load.file("foo.csv")

In [5]:
df

name 1 (object) not nullable,job 2 (object) not nullable,id 3 (object) not nullable
optimus,Leader,1
optimus,Espionage,2
bumblebee,1,3
bumblebee,3,40


<class 'optimus.engines.pandas.dataframe.PandasDataFrame'>

In [20]:
class BaseDataFrame:
    def __init__(self):
        self.df=None
    def set(self, df):
        self.df = df
    

In [31]:
# from optimus.engines.base.odataframe import BaseDataFrame
def f(df):    
    df.cols.lower()
    
client.submit(f)

In [ ]:
#server
def server(func):
    df.data= op.get_var("df")
    func(df)


In [32]:
f()

1


TypeError: Can't instantiate abstract class BaseDataFrame with abstract methods compute, debug, execute, partitions, query, sample, show

In [107]:
df.cols.scatter()

TypeError: scatter() missing 1 required positional argument: 'columns'

In [7]:
df.ascii()

name        job                 id
(object)    (object)      (object)
----------  ----------  ----------
optimus     Leader               1
optimus     Espionage            2
bumblebee   1                    3
bumblebee   3                    4


In [22]:
df.cols.quality()

args ()


{'name': {'match': 4,
  'missing': 0,
  'mismatch': 0,
  'profiler_dtype': {'dtype': 'string'}},
 'job': {'match': 4,
  'missing': 0,
  'mismatch': 0,
  'profiler_dtype': {'dtype': 'string'}},
 'id': {'match': 4,
  'missing': 0,
  'mismatch': 0,
  'profiler_dtype': {'dtype': 'int'}}}

In [23]:
df.cols.profiler_dtypes({"job":"string"})

TypeError: list indices must be integers or slices, not dict

In [ ]:
df

In [61]:
df.profile(bins=5)

{'columns': {'name': {'stats': {'match': 4,
    'missing': 0,
    'mismatch': 0,
    'profiler_dtype': {'dtype': 'string'},
    'frequency': [{'value': 'bumblebee', 'count': 2},
     {'value': 'optimus', 'count': 2}],
    'count_uniques': 2},
   'dtype': 'object'},
  'job': {'stats': {'match': 4,
    'missing': 0,
    'mismatch': 0,
    'profiler_dtype': {'dtype': 'string'},
    'frequency': [{'value': 'Leader', 'count': 1},
     {'value': '3', 'count': 1},
     {'value': 'Espionage', 'count': 1},
     {'value': '1', 'count': 1}],
    'count_uniques': 4},
   'dtype': 'object'},
  'id': {'stats': {'match': 4,
    'missing': 0,
    'mismatch': 0,
    'profiler_dtype': {'dtype': 'int'},
    'hist': [{'lower': 1.0, 'upper': 1.75, 'count': 1},
     {'lower': 1.75, 'upper': 2.5, 'count': 1},
     {'lower': 2.5, 'upper': 3.25, 'count': 1},
     {'lower': 3.25, 'upper': 4.0, 'count': 1}],
    'count_uniques': 4},
   'dtype': 'object'}},
 'name': 'foo.csv',
 'file_name': 'foo.csv',
 'summary': 

In [25]:
df.cols.hist("id",5)
# df.cols.hist("id")["hist"]["id"][0]["lower"]

args ()
args ()


{'hist': {'id': [{'lower': 1.0, 'upper': 1.75, 'count': 1},
   {'lower': 1.75, 'upper': 2.5, 'count': 1},
   {'lower': 2.5, 'upper': 3.25, 'count': 1},
   {'lower': 3.25, 'upper': 4.0, 'count': 1}]}}

In [26]:
df.cols.frequency("*",5)

{'frequency': {'name': {'values': [{'value': 'bumblebee', 'count': 2},
    {'value': 'optimus', 'count': 2}]},
  'job': {'values': [{'value': 'Leader', 'count': 1},
    {'value': '3', 'count': 1},
    {'value': 'Espionage', 'count': 1},
    {'value': '1', 'count': 1}]},
  'id': {'values': [{'value': '3', 'count': 1},
    {'value': '1', 'count': 1},
    {'value': '4', 'count': 1},
    {'value': '2', 'count': 1}]}}}

In [57]:
# df.cols.percentile("id")
df.cols.percentile("*", [0.25, 0.5, 0.75], relative_error=1)

{'name': nan,
 'job': {0.25: 1.5, 0.5: 2.0, 0.75: 2.5},
 'id': {0.25: 1.75, 0.5: 2.5, 0.75: 3.25}}

In [53]:
df.cols.names()

['name', 'job', 'id']

In [54]:
df.cols.iqr("*")

{'name': nan, 'job': 1.0, 'id': 1.5}

In [58]:
df.cols.boxplot("id")

{'id': {'mean': 2.5,
  'median': 2.5,
  'q1': 1.75,
  'q3': 3.25,
  'whisker_low': -0.5,
  'whisker_high': 5.5,
  'fliers': [],
  'label': 'id'}}

In [209]:
df.cols.min("id")

1

In [210]:
df.cols.min(["id","name"])

{'id': 1, 'name': 'bumblebee'}

In [128]:
df = df.cols.cast("id","int")

In [134]:
df

name 1 (object) not nullable,job 2 (object) not nullable,id 3 (int64) not nullable
optimus,Leader,1
optimus,Espionage,2
bumblebee,1,3
bumblebee,3,4


<class 'optimus.engines.pandas.dataframe.PandasDataFrame'>

In [168]:
df

name 1 (object) not nullable,job 2 (object) not nullable,id 3 (int64) not nullable
optimus,Leader,1
optimus,Espionage,2
bumblebee,1,3
bumblebee,3,4


<class 'optimus.engines.pandas.dataframe.PandasDataFrame'>

In [199]:
df.cols.count()

3

In [69]:
df.cols.min("id")

'1'

In [70]:
df.cols.cumsum("id")

0     1.0
1     3.0
2     6.0
3    46.0
Name: id, dtype: float64

In [71]:
df.cols.min("name")

'bumblebee'

In [72]:
df.cols.min(["id", "name"])

{'id': '1', 'name': 'bumblebee'}

In [73]:
df.agg({"id":min})

{'id': '1'}

In [74]:
df.agg({"name":min,"id":max})

{'name': 'bumblebee', 'id': '40'}

In [212]:
df.agg({"id":min}, groupby="name")

{'bumblebee': 3, 'optimus': 1}

In [217]:
print(df.cols.std("id"))
print(df.cols.min("id"))

1.2909944487358056
1


In [220]:
df.agg({"id":"std","name":min})

{'id': 1.2909944487358056, 'name': 'bumblebee'}

In [112]:
df = df.cols.cast("id","int")
# df.groupby("name").agg([min,max])
# df.groupby("name").agg({"id":min})
# df.groupby("name").agg({"id":min,"id":"mean"})
# df.groupby("name").agg(x=('id', max))

TypeError: parse_columns() takes from 2 to 6 positional arguments but 7 were given

In [75]:
df = op.load.file("foo.csv")

In [76]:
df= df.cols.cast("id","int")

In [77]:
df.cols.min("id")

1

In [78]:
df.cols.max("id")

40

In [95]:
df.cols.mode("id")

{'id': {0: 1.0, 1: 2.0, 2: 3.0, 3: 40.0}}

In [93]:
df.cols.median("id")

2.5

In [81]:
df.cols.iqr("id")

10.5

In [82]:
df.cols.mean("id")

11.5

In [83]:
df.cols.std("id")

19.017535767461215

In [84]:
df.cols.var("id")

361.6666666666667

In [89]:
df.cols.skew("id")

1.988947740397821

In [96]:
df.cols.count_zeros("id")

0

In [98]:
df.cols.count_na("id")

0

In [101]:
df.cols.count_uniques()

{'name': 2, 'job': 4, 'id': 4}

In [113]:
profiler = df.profile(bins=5)

In [114]:
profiler["columns"]

{'name': {'stats': {'match': 4,
   'missing': 0,
   'mismatch': 0,
   'profiler_dtype': {'dtype': 'string'},
   'frequency': [{'value': 'bumblebee', 'count': 2},
    {'value': 'optimus', 'count': 2}],
   'count_uniques': 2},
  'dtype': 'object'},
 'job': {'stats': {'match': 4,
   'missing': 0,
   'mismatch': 0,
   'profiler_dtype': {'dtype': 'string'},
   'frequency': [{'value': '3', 'count': 1},
    {'value': 'Leader', 'count': 1},
    {'value': 'Espionage', 'count': 1},
    {'value': '1', 'count': 1}],
   'count_uniques': 4},
  'dtype': 'object'},
 'id': {'stats': {'match': 4,
   'missing': 0,
   'mismatch': 0,
   'profiler_dtype': {'dtype': 'int'},
   'hist': [{'lower': 1.0, 'upper': 10.75, 'count': 3},
    {'lower': 10.75, 'upper': 20.5, 'count': 0},
    {'lower': 20.5, 'upper': 30.25, 'count': 0},
    {'lower': 30.25, 'upper': 40.0, 'count': 1}],
   'count_uniques': 4},
  'dtype': 'object'}}

In [118]:
profiler["columns"]["name"]

'object'

In [116]:
profiler["summary"]

{'cols_count': 3,
 'rows_count': 4,
 'dtypes_list': ['object'],
 'total_count_dtypes': 1,
 'missing_count': 0,
 'p_missing': 0.0}

In [119]:
from optimus.helpers.constants import Actions
Actions.list()

['profiler_dtype',
 'lower',
 'upper',
 'proper',
 'pad',
 'trim',
 'reverse',
 'remove',
 'left',
 'right',
 'mid',
 'replace',
 'fill_na',
 'cast',
 'is_na',
 'z_score',
 'nest',
 'unnest',
 'set',
 'string_to_index',
 'date_format',
 'index_to_string',
 'min_max_scaler',
 'max_abs_scaler',
 'apply_cols',
 'impute',
 'extract',
 'abs',
 'math',
 'variance',
 'slice',
 'clip',
 'drop',
 'keep',
 'cut',
 'to_float',
 'to_integer',
 'to_boolean',
 'to_string',
 'years',
 'append',
 'port',
 'domain',
 'domain_scheme',
 'subdomain',
 'host',
 'domain_params',
 'domain_path',
 'email_domain',
 'email_user',
 'select_row',
 'drop_row',
 'between_drop',
 'sort_row']